# Data Preprocessing

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/MSC_Intro_to_NLP_Group_Project/

In [3]:
!pip install nltk
!pip install gensim
!pip install 'transformers[torch]'
!pip install datasets
!pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [4]:
import json
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import re
import nltk
import string
from nltk import word_tokenize, pos_tag
import contractions
import pandas as pd
import spacy
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
!python -m spacy download en_core_web_sm

[nltk_data] Downloading package punkt to
[nltk_data]     /home/stud/giris1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/stud/giris1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 12.8 MB 12 kB/s  eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
# Using the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
gpu_name = torch.cuda.get_device_name(device)
print(gpu_name)

cuda
NVIDIA GeForce GTX 1080 Ti


In [6]:
# Load spaCy model outside of the function to avoid reloading it each time the function is called
nlp = spacy.load("en_core_web_sm")

In [7]:
# Function to capitalize the first letter of each sentence and proper nouns
def capitalize_first_and_proper_nouns(text):
    # Process the text using spaCy to create a Doc object
    doc = nlp(text)

    result = []

    # Iterate over the sentences in the Doc
    for sent in doc.sents:
        # Iterate over the tokens in the sentence
        for token in sent:
            # Capitalize the first letter of each sentence and proper nouns
            if token.is_sent_start or token.pos_ == 'PROPN':
                result.append(token.text.capitalize())
            else:
                result.append(token.text)

    # Rejoin the tokens into a single string
    return ' '.join(result)

# Function to remove repeated punctuations
def remove_repeated_punctuations(sentence):
    # Use regular expression to remove consecutive repeated punctuations
    cleaned_sentence = re.sub(r'(\W)\1+', r'\1', sentence)
    return cleaned_sentence

# Function to expand contractions
def expand_contractions(text):
    return contractions.fix(text)

# Define a tokenization function
def tokenize_sentences(sentences):
    return [word_tokenize(sentence) for sentence in sentences]

def fix_general_spacing(sentence):
    # Fix space before punctuation (like ' ,' to ',')
    sentence = re.sub(r'\s([,.?!:;])', r'\1', sentence)
    # Fix space after punctuation (like ' . ' to '. ')
    sentence = re.sub(r'([,.?!:;])\s', r'\1 ', sentence)
    # Fix space in contractions (like "don 't" to "don't")
    sentence = re.sub(r"\b(\w+)\s('t|'s|'m|'ll|'ve|'re|'d|n't)\b", r"\1\2", sentence)
    # Reduce multiple spaces between words to a single space
    sentence = re.sub(r'\s{2,}', ' ', sentence)
    return sentence

def preprocess(text):
    # text = text.lower()
    text = expand_contractions(text)
    text = remove_repeated_punctuations(text)
    text = capitalize_first_and_proper_nouns(text)
    text = fix_general_spacing(text)
    return text


In [8]:
# # Load the preprocessed data from the JSON file
# data_files={
#     "train":"data_train.json",
# }

# dataset = load_dataset("json", data_files=data_files)
# print(dataset)

json_file_path = "data_train.json"

with open(json_file_path, 'r') as file:
    dataset = json.load(file)


# a dataset class to load and preprocess the data

In [9]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "id": item["id"],
            "topic": item["topic"],
            "input_seq": torch.tensor(item["input_seq"]),  # Make sure this key matches the one used in numericalize_data
            "output_seq": torch.tensor(item["output_seq"]),  # Make sure this key matches the one used in numericalize_data
        }


Creating a Bi-Directional RNN Model

In [10]:
def preprocess_data(data):
    for item in data:
        item["transformation"]["informal"] = preprocess(item["transformation"]["informal"])
    return data

In [11]:
# preprocessed_data = preprocess_data(dataset)


In [12]:
def tokenize_data(data):
    for item in data:
        item["tokenized_informal"] = word_tokenize(item["transformation"]["informal"])
        item["tokenized_formal"] = word_tokenize(item["transformation"]["formal.ref0"])
    return data


In [13]:
def build_vocab(data):
    input_vocab = set()
    output_vocab = set()

    for item in data:
        input_vocab.update(item["tokenized_informal"])
        output_vocab.update(item["tokenized_formal"])

    input_vocab = {word: idx for idx, word in enumerate(input_vocab)}
    output_vocab = {word: idx for idx, word in enumerate(output_vocab)}

    return input_vocab, output_vocab


In [14]:
def pad_sequence(sequence, max_length, vocab):
    padded_sequence = [vocab[word] for word in sequence if word in vocab]
    padded_sequence += [0] * (max_length - len(padded_sequence))
    return padded_sequence

def numericalize_data(data, input_vocab, output_vocab):
    max_input_length = max(len(item["tokenized_informal"]) for item in data)
    max_output_length = max(len(item["tokenized_formal"]) for item in data)

    for item in data:
        item["input_seq"] = pad_sequence(item["tokenized_informal"], max_input_length, input_vocab)
        item["output_seq"] = pad_sequence(item["tokenized_formal"], max_output_length, output_vocab)

    return data


Defining a Bi-Directional RNN Model

In [15]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.birnn = nn.LSTM(hidden_size, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, len(output_vocab))

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.birnn(embedded)
        output = self.fc(output)
        return torch.nn.functional.log_softmax(output, dim=2)


Training function

In [16]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            inputs = batch["input_seq"].to(device)
            targets = batch["output_seq"].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs.view(-1, len(output_vocab)), targets.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")


preparing the data

In [17]:
data = preprocess_data(dataset)
data = tokenize_data(data)


In [18]:
print(data[1])

{'id': 1, 'topic': 'Family_Relationships', 'transformation': {'informal': 'Hmmm, I am a guy suffering from verbal abuse from my wife.', 'formal.ref0': 'I suffer through verbal abuse from my wife.', 'formal.ref1': '', 'formal.ref2': '', 'formal.ref3': ''}, 'tokenized_informal': ['Hmmm', ',', 'I', 'am', 'a', 'guy', 'suffering', 'from', 'verbal', 'abuse', 'from', 'my', 'wife', '.'], 'tokenized_formal': ['I', 'suffer', 'through', 'verbal', 'abuse', 'from', 'my', 'wife', '.']}


In [19]:
input_vocab, output_vocab = build_vocab(data)
data = numericalize_data(data, input_vocab, output_vocab)

In [20]:
print(data[0])

{'id': 0, 'topic': 'Family_Relationships', 'transformation': {'informal': 'Sure, it is ok, but I always have let the guy ask me.', 'formal.ref0': 'I prefer to let the guy ask me.', 'formal.ref1': '', 'formal.ref2': '', 'formal.ref3': ''}, 'tokenized_informal': ['Sure', ',', 'it', 'is', 'ok', ',', 'but', 'I', 'always', 'have', 'let', 'the', 'guy', 'ask', 'me', '.'], 'tokenized_formal': ['I', 'prefer', 'to', 'let', 'the', 'guy', 'ask', 'me', '.'], 'input_seq': [16530, 589, 7844, 41691, 2854, 589, 2753, 20797, 16830, 17080, 42810, 3802, 32836, 20071, 1269, 12076, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'output_seq': [14793, 856, 34733, 30421, 2679, 23389, 14280, 905, 8591, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [21]:
train_data = data
train_dataset = CustomDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [22]:
model = BiRNN(len(input_vocab), 256, len(output_vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [23]:
trained_model = train_model(model, train_loader, criterion, optimizer)

Epoch 1/10:   0%|                                                                    | 0/3268 [00:00<?, ?it/s]


ValueError: Expected input batch_size (1440) to match target batch_size (1568).

In [ ]:


# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# # Tokenize the informal sentences
# def preprocess_function(examples, input_field="informal", target_field="formal.ref0"):
#     inputs = [ex[input_field] for ex in examples["transformation"]]
#     targets = [ex[target_field] for ex in examples["transformation"]]

#     new_examples = tokenizer(
#         inputs, text_target=targets, max_length=64, truncation=True, padding="max_length"
#     )

#     return new_examples

# def create_multi_ref_dataset(dataset):
#   for i, target_field in enumerate(['formal.ref0', 'formal.ref1', 'formal.ref2', 'formal.ref3', 'rule_based_preprocessed']):
#     new_dataset = preprocess_function(dataset, 'informal', target_field)
#     dataset = dataset.add_column(f'labels_{i}', new_dataset['labels'])
#     if i == 0:
#       dataset = dataset.add_column('input_ids', new_dataset['input_ids'])
#       dataset = dataset.add_column('token_type_ids', new_dataset['token_type_ids'])
#       dataset = dataset.add_column('attention_mask', new_dataset['attention_mask'])

#   return dataset

# train_dataset = dataset['train'].map(
#     preprocess_function,
#     batched=True,
# )

# # tokenizer.convert_ids_to_tokens(train_dataset['input_ids'][0])


In [ ]:
print(train_dataset['input_ids'][0])

# BiRNN

In [ ]:
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim.downloader as api
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense


In [ ]:
# Load the pre-trained GloVe model
glove_model = api.load("glove-twitter-25")
nltk.download('punkt')
nltk.download('stopwords')


In [ ]:
# Load the preprocessed data from the JSON file
data_files = {
    "train": "data_train_rule_based_preprocess.json",
}

dataset = load_dataset("json", data_files=data_files)


In [ ]:
def preprocess_function(examples, input_field="informal", target_field="formal.ref0"):
    input_sequences = []
    target_sequences = []

    for ex in examples["transformation"]:
        input_sentence = ex[input_field]
        target_sentence = ex[target_field]

        # Tokenize, remove stopwords, and convert to lowercase for both input and target
        input_tokens = [token.lower() for token in word_tokenize(input_sentence) if token.isalpha()]
        input_tokens = [token for token in input_tokens if token not in stopwords.words("english")]

        target_tokens = [token.lower() for token in word_tokenize(target_sentence) if token.isalpha()]
        target_tokens = [token for token in target_tokens if token not in stopwords.words("english")]

        # Convert each word to its word embedding for both input and target
        input_embeddings = [glove_model[word] for word in input_tokens if word in glove_model]
        target_embeddings = [glove_model[word] for word in target_tokens if word in glove_model]

        input_sequences.append(input_embeddings)
        target_sequences.append(target_embeddings)

    return {"input_sequences": input_sequences, "target_sequences": target_sequences}

    # Process the data
train_dataset = dataset["train"].map(
    preprocess_function,
    batched=True,
)

# Pad sequences to have the same length
X_padded = pad_sequences(train_dataset["input_sequences"], padding='post', dtype='float32')
Y_padded = pad_sequences(train_dataset["target_sequences"], padding='post', dtype='float32')

# Reshape X_padded and Y_padded to be 3D tensors (batch_size, sequence_length, input_size)
X_padded = X_padded.reshape(X_padded.shape[0], X_padded.shape[1], -1)
Y_padded = Y_padded.reshape(Y_padded.shape[0], Y_padded.shape[1], -1)



In [ ]:
print(len(X_padded[0]))

In [ ]:
print(len(Y_padded[0]))

developing and training the model using pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np


In [ ]:
# Define the Bidirectional RNN model
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.EmbeddingBag(input_size, hidden_size, sparse=True)
        self.birnn = nn.RNN(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.birnn(embedded)
        output = self.fc(output)
        return output


In [ ]:
# Convert numpy arrays to PyTorch tensors
X_tensor = torch.from_numpy(X_padded)
Y_tensor = torch.from_numpy(Y_padded)


In [ ]:
# Create DataLoader for training and validation sets
train_dataset = TensorDataset(X_tensor, Y_tensor)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Instantiate the model
input_size = X_padded.shape[-1]
hidden_size = 64
output_size = Y_padded.shape[-1]

model = BiRNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_X, batch_Y in train_loader:
        optimizer.zero_grad()

        # Reshape the input tensor if needed
        batch_X = batch_X.view(batch_X.size(0), -1)

        output = model(batch_X)
        loss = criterion(output, batch_Y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')


# Another Try

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

# Define the BiRNN model
model = Sequential()
model.add(Embedding(input_dim=len(glove_model.index_to_key), output_dim=25, input_length=X_padded.shape[1]))
model.add(Bidirectional(LSTM(50, return_sequences=True)))
model.add(Dense(25, activation='linear'))  # Adjust the activation based on your problem

# Compile the model
model.compile(optimizer='adam', loss='mse')  # Adjust the loss based on your problem

# Display the model summary
model.summary()


# Trying with proper tokenization

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset
from torch.nn.utils.rnn import pad_sequence


# Load the preprocessed data from the JSON file
data_files = {
    "train": "data_train_rule_based_preprocess.json",
}

dataset = load_dataset("json", data_files=data_files)
print(dataset)

# Tokenizer for informal sentences
def tokenize_informal(sentence):
    return word_tokenize(sentence.lower())  # You may need to adjust this based on your specific requirements

# Tokenizer for formal sentences
def tokenize_formal(sentence):
    return word_tokenize(sentence.lower())  # You may need to adjust this based on your specific requirements

# Tokenize the informal and formal sentences
def preprocess_function(examples, input_field="informal", target_field="formal.ref0"):
    inputs = [tokenize_informal(ex[input_field]) for ex in examples["transformation"]]
    targets = [tokenize_formal(ex[target_field]) for ex in examples["transformation"]]

    return inputs, targets

train_inputs, train_targets = preprocess_function(dataset['train'])


# Convert tokens to indices
vocab = set(word for sentence in train_inputs + train_targets for word in sentence)
word_to_index = {word: index + 2 for index, word in enumerate(vocab)}
word_to_index['<pad>'] = 0
word_to_index['<unk>'] = 1
index_to_word = {index: word for word, index in word_to_index.items()}

def sentence_to_indices(sentence, word_to_index):
    return [word_to_index.get(word, word_to_index['<unk>']) for word in sentence]

train_inputs_indices = [torch.tensor(sentence_to_indices(sentence, word_to_index)) for sentence in train_inputs]
train_targets_indices = [torch.tensor(sentence_to_indices(sentence, word_to_index)) for sentence in train_targets]

# Create PyTorch Dataset and DataLoader
class CustomDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

# Update the data preparation
train_inputs_indices = [torch.tensor(sentence_to_indices(sentence, word_to_index)) for sentence in train_inputs]
train_targets_indices = [torch.tensor(sentence_to_indices(sentence, word_to_index)) for sentence in train_targets]

# Create PyTorch Dataset and DataLoader
train_dataset = CustomDataset(train_inputs_indices, train_targets_indices)
def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_padded = pad_sequence(inputs, batch_first=True)
    targets_padded = pad_sequence(targets, batch_first=True)
    return inputs_padded, targets_padded



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Bi-directional LSTM model
class BiLSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = self.fc(output[:, -1, :])  # Use the last time step's output
        return output

# Hyperparameters
vocab_size = len(word_to_index)
embedding_dim = 50
hidden_size = 64
output_size = vocab_size

# Initialize the model, loss function, and optimizer
model = BiLSTMModel(vocab_size, embedding_dim, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}")

# Save the trained model if needed
torch.save(model.state_dict(), "bilstm_model.pth")


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the JSON file into a list of dictionaries
with open("data_train_rule_based_preprocess.json", "r") as file:
    data = json.load(file)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Ensure the DataFrame has "informal" and "formal.ref0" columns
# You may need to adjust column names based on the actual structure of your data
df = df.rename(columns={"transformation": "data"})  # Assuming "transformation" contains the relevant data
df["informal"] = df["data"].apply(lambda x: x["informal"])
df["formal.ref0"] = df["data"].apply(lambda x: x["formal.ref0"])
df["rule_based_preprocessed"] = df["data"].apply(lambda x: x["rule_based_preprocessed"])

# Optional: Drop unnecessary columns
df = df[["informal", "formal.ref0", "rule_based_preprocessed"]]

# Display the DataFrame
print(df.head())



In [ ]:
# Tokenization function
def tokenize_sentence(sentence):
    return word_tokenize(sentence)

# Apply tokenization to each sentence in the DataFrame
df['informal_tokens'] = df['informal'].apply(tokenize_sentence)
df['formal_tokens'] = df['formal.ref0'].apply(tokenize_sentence)

# Split the data into training and testing sets
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define a PyTorch Dataset
class FormalDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        informal_tokens = self.data.iloc[idx]['informal_tokens']
        formal_tokens = self.data.iloc[idx]['formal_tokens']

        return {
            'informal_tokens': torch.tensor([informal_tokens]).squeeze(),
            'formal_tokens': torch.tensor([formal_tokens]).squeeze()
        }

# Create datasets and data loaders
train_dataset = FormalDataset(df)
# test_dataset = FormalDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Define the Bi-directional RNN model
class BiRNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(BiRNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.birnn = nn.GRU(embedding_dim, hidden_size, bidirectional=True)
        self.fc = nn.Linear(2 * hidden_size, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.birnn(embedded)
        output = self.fc(output)
        return output

# Hyperparameters
vocab_size = 30522
embedding_dim = 300
hidden_size = 256
learning_rate = 0.001
num_epochs = 10

# Initialize the model, loss function, and optimizer
model = BiRNNModel(vocab_size, embedding_dim, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [ ]:
# Iterate through batches of data for training
for epoch in range(num_epochs):
    for batch in train_loader:
        informal_tokens = batch['informal_tokens'].squeeze(0)  # Remove the batch dimension
        formal_tokens = batch['formal_tokens'].squeeze(0)  # Remove the batch dimension

        # Forward pass
        outputs = model(informal_tokens)

        # Compute the loss
        loss = criterion(outputs.view(-1, vocab_size), formal_tokens.view(-1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the training loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


# try again

In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
import pandas as pd

# Load the preprocessed data from the JSON file
data_files = {
    "train": "data_train_rule_based_preprocess.json",
}

# Assuming you have your data in a JSON file
with open(data_files["train"], "r") as json_file:
    data = json.load(json_file)

# Tokenize the informal sentences
def tokenize_sentences(sentences):
    tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]
    return tokenized_sentences

# Sample data
informal_sentences = [entry["transformation"]["informal"] for entry in data]
preprocessed_sentences = [entry["transformation"]["rule_based_preprocessed"] for entry in data]

# Tokenize the informal sentences
tokenized_informal = tokenize_sentences(informal_sentences)
tokenized_preprocessed = tokenize_sentences(preprocessed_sentences)

# Find the maximum sequence length in the dataset
max_informal_seq_length = max(len(sentence) for sentence in tokenized_informal)
max_preprocessed_seq_length = max(len(sentence) for sentence in tokenized_preprocessed)
print("Max Informal: ", max_informal_seq_length)
print("Max Preprocessed: ", max_preprocessed_seq_length)
# Calculate the average sequence length
avg_informal_seq_length = int(np.mean([len(sentence) for sentence in tokenized_informal]))
avg_preprocessed_seq_length = int(np.mean([len(sentence) for sentence in tokenized_preprocessed]))
print("Avg Informal: ", avg_informal_seq_length)
print("Avg Preprocessed: ", avg_preprocessed_seq_length)

# Create a vocabulary mapping words to indices
word2index = {word: index for index, word in enumerate(set(np.concatenate(tokenized_informal)))}

# Convert sentences to indices and pad sequences
indexed_sentences = [[word2index[word] for word in sentence] + [0] * (max_seq_length - len(sentence)) for sentence in tokenized_informal]
# Convert sentences to indices and pad sequences to the average length
# indexed_sentences = [[word2index[word] for word in sentence] + [0] * (avg_seq_length - len(sentence)) for sentence in tokenized_informal]



In [ ]:
print(len(indexed_sentences[0]))

In [ ]:
# Define a PyTorch Dataset class
class FormalDataset(Dataset):
    def __init__(self, indexed_sentences, formal_sentences):
        self.indexed_sentences = indexed_sentences
        self.formal_sentences = formal_sentences

    def __len__(self):
        return len(self.indexed_sentences)

    def __getitem__(self, idx):
        sample = {
            "indexed_sentence": self.indexed_sentences[idx],
            "formal_sentence": self.formal_sentences[idx],
        }
        return sample
# Create a PyTorch Dataset
formal_dataset = FormalDataset(indexed_sentences, [entry["transformation"]["formal.ref0"] for entry in data])


In [ ]:
# Create a PyTorch Dataset
formal_dataset = FormalDataset(indexed_sentences, [entry["transformation"]["formal.ref0"] for entry in data])

# Sample DataLoader with collate_fn for dynamic padding
dataloader = DataLoader(
    formal_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=lambda batch: {
        "indexed_sentence": torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(item["indexed_sentence"]) for item in batch],
            batch_first=True,
            padding_value=0,
        ),
        "formal_sentence": [item["formal_sentence"] for item in batch],
    },
)


In [ ]:
# Example of using DataLoader
for batch in dataloader:
    indexed_sentences_batch = batch["indexed_sentence"]
    formal_sentences_batch = batch["formal_sentence"]
    print(batch)
    print(indexed_sentences_batch)
    print(formal_sentences_batch)
